In [1]:
#! -*- coding: utf-8 -*-
# 微调多国语言版T5做Seq2Seq任务
# 介绍链接：kexue.fm/archives/7867
# 数据集：https://github.com/CLUEbenchmark/CLGE 中的CSL数据集
# 补充了评测指标bleu、rouge-1、rouge-2、rouge-l

from __future__ import print_function
import json
import numpy as np
from tqdm import tqdm
from bert4keras.backend import keras, K
from bert4keras.layers import Loss
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import SpTokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder
from keras.models import Model
from rouge import Rouge  # pip install rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

/home/xuehp/anaconda3/envs/t5_in_bert4keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xuehp/anaconda3/envs/t5_in_bert4keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xuehp/anaconda3/envs/t5_in_bert4keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xuehp/anacond

In [2]:

# 基本参数
max_c_len = 256
max_t_len = 32
batch_size = 16
epochs = 40

# 模型路径
folder = './mt5/'
config_path = folder + 'mt5_small_config.json'
checkpoint_path = folder + 'mt5_small/model.ckpt-1000000'
spm_path = folder + 'sentencepiece_cn.model'
keep_tokens_path = folder + 'sentencepiece_cn_keep_tokens.json'



In [3]:

def load_data(filename):
    D = []
    with open(filename, encoding='utf-8') as f:
        for l in f:
            title, content = l.strip().split('\t')
            D.append((title, content))
    return D


In [4]:

# 加载数据集
data_folder = './csl_summary_dataset/'
train_data = load_data(data_folder + 'train.tsv')
valid_data = load_data(data_folder + 'val.tsv')
test_data = load_data(data_folder + 'test.tsv')


In [5]:

# 加载分词器
tokenizer = SpTokenizer(spm_path, token_start=None, token_end='</s>')
keep_tokens = json.load(open(keep_tokens_path))



In [6]:

class data_generator(DataGenerator):
    """数据生成器
    """

    def __iter__(self, random=False):
        batch_c_token_ids, batch_t_token_ids = [], []
        for is_end, (title, content) in self.sample(random):
            c_token_ids, _ = tokenizer.encode(content, maxlen=max_c_len)
            t_token_ids, _ = tokenizer.encode(title, maxlen=max_t_len)
            batch_c_token_ids.append(c_token_ids)
            batch_t_token_ids.append([0] + t_token_ids)
            if len(batch_c_token_ids) == self.batch_size or is_end:
                batch_c_token_ids = sequence_padding(batch_c_token_ids)
                batch_t_token_ids = sequence_padding(batch_t_token_ids)
                yield [batch_c_token_ids, batch_t_token_ids], None
                batch_c_token_ids, batch_t_token_ids = [], []


In [7]:

class CrossEntropy(Loss):
    """交叉熵作为loss，并mask掉输入部分
    """

    def compute_loss(self, inputs, mask=None):
        y_true, y_pred = inputs
        y_true = y_true[:, 1:]  # 目标token_ids
        y_mask = K.cast(mask[1], K.floatx())[:, :-1]  # 解码器自带mask
        y_pred = y_pred[:, :-1]  # 预测序列，错开一位
        loss = K.sparse_categorical_crossentropy(y_true, y_pred)
        loss = K.sum(loss * y_mask) / K.sum(y_mask)
        return loss



In [8]:

t5 = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    keep_tokens=keep_tokens,
    model='t5.1.1',
    return_keras_model=False,
    name='T5',
)

encoder = t5.encoder
decoder = t5.decoder
model = t5.model
model.summary()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder-Input-Token (InputLayer (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 512)    16690176    Encoder-Input-Token[0][0]        
__________________________________________________________________________________________________
Encoder-Embedding-Dropout (Drop (None, None, 512)    0           Embedding-Token[0][0]            
__________________________________________________________________________________________________
Encoder-Transformer-0-MultiHead (None, None, 512)    512         Encoder-Embedding-Dropout[0][0] 

In [9]:

output = CrossEntropy(1)([model.inputs[1], model.outputs[0]])

model = Model(model.inputs, output)
model.compile(optimizer=Adam(2e-4))



/home/xuehp/anaconda3/envs/t5_in_bert4keras/lib/python3.6/site-packages/keras/engine/training_utils.py:819: UserWarning: Output cross_entropy_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to cross_entropy_1.
  'be expecting any data to be passed to {0}.'.format(name))


In [10]:

class AutoTitle(AutoRegressiveDecoder):
    """seq2seq解码器
    """

    @AutoRegressiveDecoder.wraps(default_rtype='probas')
    def predict(self, inputs, output_ids, states):
        c_encoded = inputs[0]
        return decoder.predict([c_encoded, output_ids])[:, -1]

    def generate(self, text, topk=1):
        c_token_ids, _ = tokenizer.encode(text, maxlen=max_c_len)
        c_encoded = encoder.predict(np.array([c_token_ids]))[0]
        output_ids = self.beam_search([c_encoded], topk)  # 基于beam search
        return tokenizer.decode([int(i) for i in output_ids])



In [11]:

# 注：T5有一个很让人不解的设置，它的<bos>标记id是0，即<bos>和<pad>其实都是0
autotitle = AutoTitle(start_id=0, end_id=tokenizer._token_end_id, maxlen=32)



In [12]:

class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """

    def __init__(self):
        self.rouge = Rouge()
        self.smooth = SmoothingFunction().method1
        self.best_bleu = 0.

    def on_epoch_end(self, epoch, logs=None):
        metrics = self.evaluate(valid_data)  # 评测模型
        if metrics['bleu'] > self.best_bleu:
            self.best_bleu = metrics['bleu']
            model.save_weights('./best_model.weights')  # 保存模型
        metrics['best_bleu'] = self.best_bleu
        print('valid_data:', metrics)

    def evaluate(self, data, topk=1):
        total = 0
        rouge_1, rouge_2, rouge_l, bleu = 0, 0, 0, 0
        for title, content in tqdm(data):
            total += 1
            title = ' '.join(title).lower()
            pred_title = ' '.join(autotitle.generate(content, topk)).lower()
            print(title)
            print(pred_title)
            print(content)
            print('')
            if pred_title.strip():
                scores = self.rouge.get_scores(hyps=pred_title, refs=title)
                rouge_1 += scores[0]['rouge-1']['f']
                rouge_2 += scores[0]['rouge-2']['f']
                rouge_l += scores[0]['rouge-l']['f']
                bleu += sentence_bleu(
                    references=[title.split(' ')],
                    hypothesis=pred_title.split(' '),
                    smoothing_function=self.smooth
                )
        rouge_1 /= total
        rouge_2 /= total
        rouge_l /= total
        bleu /= total
        return {
            'rouge-1': rouge_1,
            'rouge-2': rouge_2,
            'rouge-l': rouge_l,
            'bleu': bleu,
        }


In [13]:
# 加载模型，评测验证集

model.load_weights('./best_model.weights')

evaluator = Evaluator()
metrics = evaluator.evaluate(valid_data)  # 评测模型
print(metrics)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 1/1000 [00:03<56:01,  3.37s/it]

超 立 方 体 网 络 的 3 路 结 构 连 通 度 及 子 结 构 连 通 度
超 立 方 体 网 络 结 构 连 通 性 和 子 结 构 连 通 性 评 价
针对以超立方体网络为蓝本的多处理机系统的可靠性和容错能力的精准度量问题,结合多处理机系统遭受计算机病毒攻击时常常发生结构性故障的特点,研究了n维超立方体网络的结构连通性和子结构连通性评价问题。首先,使用构造n维超立方体网络的3路结构割的方法得到其3路结构连通度的一个上界;然后,使用构造n维超立方体网络的3路子结构集的等价变换或约简变换的方法,得到其3路结构子连通度的一个下界;最后,利用任意网络的3路结构连通度不小于3路子结构连通度的性质,证实了超立方体网络的3路结构连通度和子结构连通度均为该超立方体网络维数的一半。这一结果表明,在3路结构故障模型下,破坏敌方以超立方体网络为底层拓扑的多处理系统至少需要攻击该系统中维数一半的3路结构或子结构。



  0%|          | 2/1000 [00:03<42:20,  2.55s/it]

基 于 惩 罚 误 差 矩 阵 的 同 步 预 测 无 线 体 域 网 节 能 方 法
基 于 惩 罚 误 差 矩 阵 的 无 线 体 域 网 自 适 应 三 次 指 数 平 滑 算 法
针对传统无线体域网(WBAN)预测模型对感知数据预测精度低、计算量大、能耗高的问题,提出一种基于惩罚误差矩阵的自适应三次指数平滑算法。首先在感知节点与路由节点之间建立轻量级预测模型,其次采用地毯式搜索方式对预测模型进行参数优化处理,最后采用惩罚误差矩阵对预测模型参数作进一步的细粒化处理。实验结果表明,与Zig Bee协议相比,在1000时隙范围内,所提方法可节省12%左右的能量;而采用惩罚误差矩阵与地毯式搜索方式相比,预测精度提高了3. 306%。所提方法在有效降低计算复杂度的同时能进一步降低WBAN的能耗。



...

小 型 有 缆 无 人 水 下 机 器 人 控 制 系 统 设 计
有 缆 型 无 人 水 下 机 器 人 的 控 制 系 统 设 计
针对有缆型无人水下机器人(ROV,Remote Operated Vehicle),本文从控制算法、传感器数据采集、任务调度方面设计了一种控制系统.首先,设计了模糊自适应串级PID控制器用于ROV的运动与姿态控制,弥补了传统控制算法的不足,控制性能得到提高.其次,采用分布式子系统采集处理传感器数据,提高了传感数据更新速率.最后,在主微控制器(MCU,Micro Con-trol Unit)中采用嵌入式实时操作系统FreeRTOS进行任务调度,系统的实时性得到增强.为了验证该控制系统方案的可行性,搭建实验环境并进行了实验,实验结果表明该系统可以对ROV进行良好的控制.



100%|█████████▉| 999/1000 [07:00<00:00,  2.51it/s]

飞 行 器 软 轴 滚 珠 检 测 技 术 研 究
基 于 图 像 形 态 学 的 飞 行 器 滚 珠 自 动 识 别 算 法
为解决飞行器滚珠式软轴的滚珠检测中使用传统人工或者半自动方法带来的大误差,长耗时等缺点,对飞行器软轴滚珠检测技术进行了研究,设计了一种基于图像形态学的滚珠自动识别算法.首先在实时成像中引入窗技术以提高图像的动态范围,再针对软轴图像过长的特点设计了一次成图技术,最后通过实验比较了多种主流图像分割方法,并据此开发了适用于滚珠式软轴的基于图像形态学的滚珠自动识别算法.实验结果表明,与传统检测方法相比,使用新方法后缺漏滚珠识别率提高了8.4%,计数分析时间也缩短为原先的3. 5%,更好的满足了工业检测要求.



100%|██████████| 1000/1000 [07:01<00:00,  2.37it/s]

i - v e c t o r 聚 类 字 典 及 注 意 力 机 制 框 架 的 说 话 人 自 适 应
一 种 基 于 i - v e c t o r 聚 类 字 典 及 注 意 力 机 制 的 自 适 应 框 架
近些年来,语音识别任务中的说话人自适应技术在实际工程中得到广泛应用.基于i-vector的说话人自适应是其中最为重要的一种,但是提取i-vector需要用到整句话的信息,并不能用于线上的自适应.因此,本文设计了一种基于i-vector聚类字典及注意力机制的自适应框架,测试时能够在不提取i-vector和不进行二遍解码的前提下快速实现线上自适应,并且该框架具有灵活性优和可扩展性好的优点,能够方便的用于其他类型的自适应,如地域自适应和性别自适应.在Switchboard任务上,实验结果表明我们提出的框架在不同的声学模型上相对于基线均有性能提升,并且通过说话人识别任务进一步证明了该方法的合理性.

{'rouge-1': 0.6745963715335307, 'rouge-2': 0.5690859850235804, 'rouge-l': 0.6541478671387893, 'bleu': 0.478420268425949}
